In [1]:
%%capture
!pip install spacy
!pip install PyPDF2
!pip install serpapi
!pip install chromadb
!pip install networkx
!pip install unidecode
!pip install langchain
!pip install fuzzywuzzy
!pip install python-docx
!pip install python-Levenshtein
!pip install langchain-community
!pip install sentence-transformers
!python -m spacy download es_core_news_md
!pip install llama-index-embeddings-huggingface==0.1.1 sentence-transformers==2.3.1 pypdf==4.0.1 langchain==0.1.7 python-decouple==3.8 llm-templates llama-index-readers-file

# Obtención y Carga de Datos

In [2]:
import gdown
import os

# Link con archivos sobre NLP
url = 'https://drive.google.com/drive/folders/1x_DXa3qQZtSaSGjXa3ntycesMABwn2eR?usp=sharing'
output_folder = 'Archivos_necesarios'

# Verifica si la carpeta ya existe
if os.path.exists(output_folder):
    print(f"La carpeta '{output_folder}' ya existe. No se descargará nuevamente.")
else:
    # Descarga la carpeta
    gdown.download_folder(url, quiet=True, output=output_folder)
    print(f"La carpeta '{output_folder}' fue descargada con éxito.")


La carpeta 'Archivos_necesarios' fue descargada con éxito.


In [3]:
import pandas as pd
import pickle

# Carga de PDFs
unidad_1 = '/content/Archivos_necesarios/Unidad 1 - Extracción y Procesamiento de Texto.pdf'
unidad_2 = '/content/Archivos_necesarios/Unidad 2 - Representación Vectorial de Texto.pdf'
unidad_3 = '/content/Archivos_necesarios/Unidad 3 - Procesamiento del Lenguaje.pdf'
unidad_4 = '/content/Archivos_necesarios/Unidad 4 - Arquitecturas de Modelos de Lenguaje.pdf'
unidad_5 = '/content/Archivos_necesarios/Unidad 5 - Almacenamiento y Representación del Conocimiento.pdf'
unidad_6 = '/content/Archivos_necesarios/Unidad 6 - Chatbots y Sistemas de Diálogo.pdf'
unidad_7 = '/content/Archivos_necesarios/Unidad 7 - Agentes Autónomos y Sistemas Inteligentes - 2024.pdf'

pdfs = [unidad_1, unidad_2, unidad_3, unidad_4, unidad_5, unidad_6, unidad_7]

# Carga del archivo CSV
ejercicios = pd.read_csv('/content/Archivos_necesarios/ejercicios_nlp_con_embeddings.csv')

# Carga del grafo
with open("/content/Archivos_necesarios/grafo_nlp.pickle", "rb") as f:
    G = pickle.load(f)

# Preprocesamiento de texto, extracción de PDF y split de datos con LangChain

In [4]:
import re
import nltk
import PyPDF2
from unidecode import unidecode
from langchain.text_splitter import RecursiveCharacterTextSplitter


def extract_text_from_pdf(pdf_path):
    # Extrae el texto de un archivo PDF
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text

def process_pdf_text(pdf_path):
    # Procesa el texto extraído del PDF
    text = extract_text_from_pdf(pdf_path)

    # Convierte todo el texto a minúsculas para uniformidad
    text = text.lower()

    # Elimina todos los caracteres que no sean letras, números o espacios
    # Esto ayuda a estandarizar el texto y eliminar puntuación y símbolos
    return text

def split_text_with_langchain(text, chunk_size=500, chunk_overlap=50):
    # Divide el texto en fragmentos más pequeños
    # chunk_size: define el tamaño máximo de cada fragmento en caracteres
    # chunk_overlap: determina cuántos caracteres se superponen entre fragmentos
    #                esto ayuda a mantener el contexto entre fragmentos
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    return text_splitter.split_text(text)

# Crear un diccionario global para almacenar los fragmentos sin stopwords
global fragments_dict
fragments_dict = {}

# Dividir todos los PDFs y procesa los fragmentos para eliminar stopwords
for i, pdf_path in enumerate(pdfs, 1):
    processed_text = process_pdf_text(pdf_path)
    fragments = split_text_with_langchain(processed_text)

    # Almacenar los fragmentos sin stopwords en el diccionario global
    fragments_dict[f'unidad_{i}'] = fragments

    print(f"\nFragmentos de la Unidad {i}:")
    for j, fragment in enumerate(fragments, 1):
        # Muestra una vista previa de cada fragmento (primeros 50 caracteres)
        print(f"Fragmento {j}: {fragment[:50]}...")

# Un chunk_size mayor dará fragmentos más largos, lo que puede ser útil para mantener más contexto
# Un chunk_overlap mayor aumentará la superposición, ayudando a preservar la continuidad entre fragmentos


Fragmentos de la Unidad 1:
Fragmento 1: unidad 1 - extracción y procesamiento de texto
1. ...
Fragmento 2: de las que se puede extraer texto:
•documentos de ...
Fragmento 3: de noticias, foros de discusión, wikipedia y más.
...
Fragmento 4: electrónico, así como los asuntos y los encabezado...
Fragmento 5: extraídas para análisis de texto.
•literatura: los...
Fragmento 6: datos.
formatos y fuentes de texto
en el mundo de ...
Fragmento 7: semi-estructurados y no estructurados, y pueden es...
Fragmento 8: formato de texto, como negrita, cursiva, colores, ...
Fragmento 9: contenido del archivo y lo almacenará en la variab...
Fragmento 10: codificación diferente, como iso-8859-1 o latin-1
...
Fragmento 11: estuviera codificado en latin-1, usaríamos encodin...
Fragmento 12: utf-8:la cadena en esa codificación posee una long...
Fragmento 13: codificarse, y además su codificación puede cambia...
Fragmento 14: encabezados, enlaces, imágenes, listas, tablas, et...
Fragmento 15: <!
>
<
>
<
>
<


# Embeddings

In [5]:
from transformers import BertModel, BertTokenizer
import torch
import numpy as np

# Cargar el modelo BERT en español ya que todos nuestros datos estan en español
modelo_es = BertModel.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')
tokenizador_es = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')

# Función que obtiene embeddings para cada texto
def obtener_embeddings(fragmentos):
    embeddings = []
    for fragmento in fragmentos:
        tokens = tokenizador_es(fragmento, truncation=True, padding=True, return_tensors='pt', max_length=512)
        with torch.no_grad():
            outputs = modelo_es(**tokens)
        embedding_vector = outputs.last_hidden_state.mean(dim=1).squeeze()
        embeddings.append(embedding_vector.tolist())
    return embeddings

# Diccionario para almacenar los embeddings
embeddings_dict = {}

# Obtener embeddings para cada unidad en el diccionario fragments_sin_stopwords_dict
for unidad, fragmentos in fragments_dict.items():
    embeddings = obtener_embeddings(fragmentos)
    embeddings_dict[unidad] = embeddings

    print(f"\nEmbeddings de los fragmentos de {unidad}:")
    print(np.array(embeddings).shape)

# Acedemos a la los embeddings de la unidad 1 a modo de ejemplo para ver como queda
if 'unidad_1' in embeddings_dict:
    print("\nEjemplo de embeddings para unidad_1:")
    print(np.array(embeddings_dict['unidad_1']))

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]


Embeddings de los fragmentos de unidad_1:
(179, 768)

Embeddings de los fragmentos de unidad_2:
(167, 768)

Embeddings de los fragmentos de unidad_3:
(185, 768)

Embeddings de los fragmentos de unidad_4:
(149, 768)

Embeddings de los fragmentos de unidad_5:
(186, 768)

Embeddings de los fragmentos de unidad_6:
(187, 768)

Embeddings de los fragmentos de unidad_7:
(67, 768)

Ejemplo de embeddings para unidad_1:
[[ 0.05367143  0.13624236 -0.31646827 ... -0.249538    0.16316204
   0.06009699]
 [-0.20899178  0.17416608 -0.28842649 ...  0.01384886  0.10865366
   0.35636562]
 [-0.27344424  0.12047482 -0.46065781 ...  0.03319148 -0.12157888
   0.09256357]
 ...
 [ 0.06931979  0.19641548 -0.27520782 ... -0.25259843 -0.30628526
  -0.11172235]
 [-0.10986084  0.2881462  -0.41449106 ... -0.09863464 -0.16996282
   0.29158288]
 [-0.47995216  0.12194171 -0.56929469 ... -0.48056489  0.16987181
   0.00934488]]


# Chromadb

In [6]:
import chromadb
import numpy as np

# Crear el cliente y la colección de Chroma
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="teoria")

# Unir todos los fragmentos en una sola lista
todos_los_fragmentos = []
todos_los_embeddings = []
ids = []

for unidad, fragmentos in fragments_dict.items():
    todos_los_fragmentos.extend(fragmentos)
    todos_los_embeddings.extend(embeddings_dict[unidad])
    ids.extend([f'{unidad}_id{i+1}' for i in range(len(fragmentos))])

# Comprobamos la cantidad total de fragmentos
print(f"Número total de fragmentos: {len(todos_los_fragmentos)}")

# Comprobamos que tenemos la misma cantidad de embeddings que de fragmentos
print(f"Número total de embeddings: {len(todos_los_embeddings)}")

# Comprobamos que tenemos la misma cantidad de IDs que de fragmentos
print(f"Número total de IDs: {len(ids)}")

# Convertimos los embeddings a una lista de listas
embeddings_lista = [embedding.tolist() if isinstance(embedding, np.ndarray) else embedding for embedding in todos_los_embeddings]

# Agregamos los embeddings a chromaDB
collection.add(
    embeddings=embeddings_lista,
    documents=todos_los_fragmentos,
    ids=ids
)

print(f"Se han agregado {len(todos_los_fragmentos)} fragmentos a la colección de ChromaDB.")

Número total de fragmentos: 1120
Número total de embeddings: 1120
Número total de IDs: 1120
Se han agregado 1120 fragmentos a la colección de ChromaDB.


# Modelo de clasificación (PDF, CSV o Grafo)

###### .

In [7]:
# Preparar los datos de entrenamiento
prompts = [
    # CSV (ejercicios)
    "Necesito una práctica sobre extracción de texto",
    "Muestra un ejercicio de OCR",
    "Busco un problema de dificultad media sobre procesamiento de lenguaje natural",
    "¿Hay algún ejercicio sobre tokenización?",
    "Quiero practicar con un ejercicio de análisis de sentimientos",
    "Necesito un ejercicio práctico sobre clasificación de textos",
    "Dame un problema para resolver sobre vectorización de palabras",
    "Busco un ejercicio difícil de reconocimiento de entidades nombradas",
    "¿Tienes algún ejercicio sobre stemming y lematización?",
    "Proporciona un ejercicio de preprocesamiento de texto",
    "Dame una tarea práctica de normalización de texto",
    "Necesito un ejercicio para practicar la eliminación de stopwords",
    "¿Hay algún problema sobre construcción de n-gramas?",
    "Quiero un ejercicio de análisis de frecuencia de palabras",
    "Busco una práctica de codificación one-hot para texto",
    "Dame un ejercicio para implementar TF-IDF",
    "Necesito una tarea sobre bag-of-words",
    "¿Tienes algún problema práctico de parsing?",
    "Proporciona un ejercicio de etiquetado POS",

    # Grafo (conceptos)
    "Explícame la relación entre tokenización y análisis sintáctico",
    "¿Cómo se conecta el procesamiento de lenguaje natural con el aprendizaje automático?",
    "Muestra el camino de aprendizaje para dominar NLP",
    "¿Qué conceptos están relacionados con el análisis de sentimientos?",
    "Describe la estructura de un pipeline de NLP",
    "¿Cuáles son los componentes principales del procesamiento de lenguaje natural?",
    "Explica la conexión entre word embeddings y redes neuronales",
    "¿Cómo se relacionan la lingüística computacional y el NLP?",
    "Muestra los prerequisitos para aprender procesamiento de lenguaje natural",
    "¿Qué técnicas de deep learning se aplican en NLP?",
    "Explica el concepto de parsing en NLP",
    "¿Cuál es la diferencia entre procesamiento de lenguaje natural y comprensión del lenguaje natural?",
    "Describe los diferentes niveles de análisis en NLP",
    "¿Qué papel juega la semántica en el procesamiento de lenguaje natural?",
    "Explica el concepto de desambiguación en NLP",
    "¿Cómo se relaciona la teoría de la información con el NLP?",
    "Describe los desafíos del procesamiento de lenguaje natural",
    "¿Qué es la generación de lenguaje natural y cómo se relaciona con NLP?",
    "Explica el concepto de modelo de lenguaje en NLP",
    "¿Cuál es la importancia de la pragmática en el procesamiento de lenguaje natural?",

    # ChromaDB (teoría)
    "Encuentra información detallada sobre el funcionamiento de BERT",
    "Busca fragmentos que expliquen la arquitectura Transformer",
    "Necesito teoría profunda sobre el modelo GPT",
    "Dame información detallada sobre técnicas avanzadas de análisis de sentimientos",
    "Busco explicaciones extensas sobre el funcionamiento de word2vec",
    "Encuentra fragmentos que describan en detalle el proceso de named entity recognition",
    "Necesito información completa sobre técnicas de resumen automático de texto",
    "Busca teoría detallada sobre modelos de lenguaje neuronales",
    "Dame explicaciones profundas sobre la traducción automática neuronal",
    "Encuentra fragmentos que describan técnicas avanzadas de question answering",
    "Busca información detallada sobre el funcionamiento de ELMo",
    "Necesito una explicación profunda del mecanismo de atención en NLP",
    "Encuentra fragmentos que describan en detalle las redes neuronales recurrentes en NLP",
    "Dame información completa sobre el proceso de fine-tuning en modelos de lenguaje",
    "Busco explicaciones detalladas sobre la arquitectura de las redes LSTM",
    "Encuentra teoría profunda sobre el uso de transformers en NLP",
    "Necesito información extensa sobre técnicas de transfer learning en procesamiento de lenguaje",
    "Busca fragmentos que expliquen en detalle el funcionamiento de los autoencoders en NLP",
    "Dame explicaciones detalladas sobre las técnicas de regularización en modelos de lenguaje",
    "Encuentra información completa sobre la evaluación de modelos de NLP",

    # Más ejemplos mixtos
    "¿Cómo puedo implementar un clasificador de texto básico?",
    "Explícame la diferencia entre supervisado y no supervisado en NLP",
    "Necesito un ejercicio para practicar web scraping de texto",
    "Busco información detallada sobre el funcionamiento de SpaCy",
    "Dame un ejemplo de cómo usar regex en procesamiento de texto",
    "¿Puedes explicarme qué es transfer learning en el contexto de NLP?",
    "Quiero un ejercicio para practicar la creación de un chatbot simple",
    "Encuentra información sobre las últimas tendencias en NLP",
    "Necesito un ejercicio para practicar la generación de texto con GPT",
    "Explícame en detalle cómo funciona la atención en los modelos Transformer",
    "¿Cómo se implementa un sistema de recomendación basado en texto?",
    "Dame ejemplos de aplicaciones de NLP en el mundo real",
    "Necesito información sobre cómo manejar textos multilingües en NLP",
    "Explícame las diferencias entre word2vec, GloVe y FastText",
    "¿Cómo se puede usar NLP para el análisis de redes sociales?",
    "Dame un ejercicio para practicar la detección de temas en un corpus de texto",
    "Busco información sobre el uso de grafos de conocimiento en NLP",
    "¿Cómo se puede implementar un corrector ortográfico usando NLP?",
    "Necesito un ejercicio para practicar la generación de resúmenes de texto",
    "Explícame cómo funciona la búsqueda semántica en NLP",
]

categorias = [
    "csv", "csv", "csv", "csv",
    "csv", "csv", "csv", "csv", "csv",
    "csv", "csv", "csv", "csv", "csv",
    "csv", "csv", "csv", "csv", "csv",
    "grafo", "grafo", "grafo", "grafo", "grafo",
    "grafo", "grafo", "grafo", "grafo", "grafo",
    "grafo", "grafo", "grafo", "grafo", "grafo",
    "grafo", "grafo", "grafo", "grafo", "grafo",
    "chromadb", "chromadb", "chromadb", "chromadb", "chromadb",
    "chromadb", "chromadb", "chromadb", "chromadb", "chromadb",
    "chromadb", "chromadb", "chromadb", "chromadb", "chromadb",
    "chromadb", "chromadb", "chromadb", "chromadb", "chromadb",
    "csv", "grafo", "csv", "chromadb", "csv",
    "chromadb", "csv", "chromadb", "csv", "chromadb",
    "csv", "grafo", "chromadb", "chromadb", "grafo",
    "csv", "grafo", "csv", "csv", "chromadb",
]

###### .

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.metrics import classification_report, make_scorer, accuracy_score
import joblib

# Preparación de datos
X = prompts
y = categorias

# Crear un pipeline
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(max_features=20)),  # Usar menos características
    ('clf', MultinomialNB())  # Naive Bayes, un modelo simple
])

# Configurar la validación cruzada dejando uno fuera
cv = LeaveOneOut()

# Realizar validación cruzada
scores = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy')

print("Puntuaciones de validación cruzada:", scores)
print("Precisión media: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# Función para obtener predicciones de validación cruzada
def get_cv_predictions(model, X, y, cv):
    y_pred = []
    for train_index, test_index in cv.split(X):
        X_train, X_test = [X[i] for i in train_index], [X[i] for i in test_index]
        y_train, y_test = [y[i] for i in train_index], [y[i] for i in test_index]
        model.fit(X_train, y_train)
        y_pred.extend(model.predict(X_test))
    return y_pred

# Obtener predicciones de validación cruzada
y_pred = get_cv_predictions(pipeline, X, y, cv)

# Imprimir informe de clasificación
print("\nInforme de clasificación:")
print(classification_report(y, y_pred))

# Entrenar el modelo final con todos los datos
pipeline.fit(X, y)

# Guardar el pipeline para uso futuro
joblib.dump(pipeline, 'pipeline_clasificacion.joblib')

# Función para predecir nuevos prompts
def predecir_categoria(prompt):
    return pipeline.predict([prompt])[0]

# Vemos un ejemplo para ver como queda
nuevo_prompt = "Dame un ejercicio de codificación de archivos"
categoria_predicha = predecir_categoria(nuevo_prompt)
print(f"\nPara el prompt: '{nuevo_prompt}'")
print(f"La categoría predicha es: {categoria_predicha}")

Puntuaciones de validación cruzada: [1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0.
 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1.
 0. 1. 1. 0. 0. 1. 0.]
Precisión media: 0.77 (+/- 0.84)

Informe de clasificación:
              precision    recall  f1-score   support

    chromadb       0.71      0.74      0.73        27
         csv       0.89      0.86      0.87        28
       grafo       0.71      0.71      0.71        24

    accuracy                           0.77        79
   macro avg       0.77      0.77      0.77        79
weighted avg       0.77      0.77      0.77        79


Para el prompt: 'Dame un ejercicio de codificación de archivos'
La categoría predicha es: csv


# Clasificador basado en un modelo entrenado con ejemplos y embeddings (Unidad 3)

In [27]:
import numpy as np
from scipy.spatial.distance import cosine
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
from functools import lru_cache

# Cargar el modelo
pipeline = joblib.load('pipeline_clasificacion.joblib')

# Inicializar el vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Definir el umbral de similitud
UMBRAL_SIMILITUD = 0.6

# Precalcular vectores TF-IDF para documentos conocidos
def precalcular_tfidf():
    global csv_vectors, grafo_vectors
    csv_vectors = tfidf_vectorizer.fit_transform(ejercicios['Enunciado'])
    grafo_vectors = tfidf_vectorizer.transform(list(G.nodes()))

precalcular_tfidf()

# Función para predecir la categoría
@lru_cache(maxsize=1000)
def predecir_categoria(prompt):
    return pipeline.predict([prompt])[0]

# Función para calcular la similitud coseno usando TF-IDF
def tfidf_cosine_similarity(prompt_vector, doc_vector):
    return 1 - cosine(prompt_vector.toarray()[0], doc_vector.toarray()[0])

# Función para encontrar la respuesta más apropiada
def encontrar_respuesta(prompt, categoria):
    prompt_vector = tfidf_vectorizer.transform([prompt])

    if categoria == 'csv':
        similitudes = [tfidf_cosine_similarity(prompt_vector, doc_vector) for doc_vector in csv_vectors]
        max_similitud = max(similitudes)
        if max_similitud >= UMBRAL_SIMILITUD:
            indice_mas_cercano = np.argmax(similitudes)
            return ejercicios.iloc[indice_mas_cercano]['Enunciado']
        else:
            return f"No se encontró información relevante sobre '{prompt}' en la categoría CSV."

    elif categoria == 'grafo':
        similitudes = [tfidf_cosine_similarity(prompt_vector, doc_vector) for doc_vector in grafo_vectors]
        max_similitud = max(similitudes)
        if max_similitud >= UMBRAL_SIMILITUD:
            indice_mas_cercano = np.argmax(similitudes)
            return list(G.nodes())[indice_mas_cercano]
        else:
            return f"No se encontró información relevante sobre '{prompt}' en la categoría Grafo."

    elif categoria == 'chromadb':
        resultados = collection.query(
            query_texts=[prompt],
            n_results=1
        )
        if resultados['documents'] and resultados['distances'][0][0] <= (1 - UMBRAL_SIMILITUD):
            return resultados['documents'][0][0]
        else:
            return f"No se encontró información relevante sobre '{prompt}' en ChromaDB."

# Función principal para procesar el prompt del usuario
def procesar_prompt(prompt):
    categoria = predecir_categoria(prompt)
    respuesta = encontrar_respuesta(prompt, categoria)
    return categoria, respuesta

# Función para procesar múltiples prompts en lotes
def procesar_prompts_en_lote(prompts):
    categorias = [predecir_categoria(prompt) for prompt in prompts]
    respuestas = [encontrar_respuesta(prompt, categoria) for prompt, categoria in zip(prompts, categorias)]
    return list(zip(categorias, respuestas))

In [28]:
prompt_usuario = "Dame un ejercicio de codificación de archivos"
categoria, respuesta = procesar_prompt(prompt_usuario)

print(f"Prompt del usuario: '{prompt_usuario}'")
print(f"Categoría predicha: {categoria}")
print(f"Respuesta encontrada: {respuesta}")

Prompt del usuario: 'Dame un ejercicio de codificación de archivos'
Categoría predicha: csv
Respuesta encontrada: No se encontró información relevante sobre 'Dame un ejercicio de codificación de archivos' en la categoría CSV.


In [29]:
prompt_usuario = "Dame un ejercicio de audio"
categoria, respuesta = procesar_prompt(prompt_usuario)

print(f"Prompt del usuario: '{prompt_usuario}'")
print(f"Categoría predicha: {categoria}")
print(f"Respuesta encontrada: {respuesta}")

Prompt del usuario: 'Dame un ejercicio de audio'
Categoría predicha: csv
Respuesta encontrada: Obtenga texto de un archivo de audio.


In [30]:
prompt_usuario = "¿Cómo podría obtener texto de un archivo Word utilizando Python?"
categoria, respuesta = procesar_prompt(prompt_usuario)

print(f"Prompt del usuario: '{prompt_usuario}'")
print(f"Categoría predicha: {categoria}")
print(f"Respuesta encontrada: {respuesta}")

Prompt del usuario: '¿Cómo podría obtener texto de un archivo Word utilizando Python?'
Categoría predicha: csv
Respuesta encontrada: Obtenga texto de un archivo word.


# Clasificador basado en LLM (Unidad 6)


In [12]:
import pandas as pd
import networkx as nx
import chromadb
import requests
from decouple import config
import sqlite3
from time import time
from google.colab import userdata

# Función para consultar el grafo
def query_graph(prompt):
    relevant_nodes = []
    for node in G.nodes():
        if any(keyword in node.lower() for keyword in prompt.lower().split()):
            relevant_nodes.append(node)
            relevant_nodes.extend(list(G.neighbors(node)))

    context = ""
    for node in set(relevant_nodes):
        context += f"Concepto: {node}\n"
        context += f"Definición: {G.nodes[node].get('definicion', 'No disponible')}\n"
        for neighbor in G.neighbors(node):
            edge_data = G.get_edge_data(node, neighbor)
            context += f"- {edge_data['relacion']} {neighbor}\n"
        context += "\n"

    return context

# Función para consultar la base de datos tabular
def query_tabular(prompt):
    conn = sqlite3.connect(':memory:')
    ejercicios.to_sql('ejercicios', conn, index=False)

    cursor = conn.cursor()
    query = f"""
    SELECT Unidad, Tema, Tipo_Ejercicio, Enunciado, Dificultad
    FROM ejercicios
    WHERE Enunciado LIKE '%{prompt}%'
    LIMIT 5
    """

    results = cursor.execute(query).fetchall()
    conn.close()

    context = ""
    for row in results:
        context += f"Unidad: {row[0]}, Tema: {row[1]}, Tipo: {row[2]}, Dificultad: {row[4]}\n"
        context += f"Enunciado: {row[3]}\n\n"

    return context

# Función para consultar ChromaDB
def query_chromadb(prompt):
    results = collection.query(
        query_texts=[prompt],
        n_results=3
    )

    context = ""
    for doc in results['documents'][0]:
        context += doc + "\n\n"

    return context

def zephyr_chat_template(messages):
    prompt = ""
    for message in messages:
        if message["role"] == "system":
            prompt += f"{message['content']}\n"
        elif message["role"] == "user":
            prompt += f"Usuario: {message['content']}\n"
        elif message["role"] == "assistant":
            prompt += f"Asistente: {message['content']}\n"
    prompt += "Asistente: "
    return prompt

def prepare_prompt(query_str: str, context_str: str = ""):
    text_qa_prompt_tmpl = (
        "Información de contexto:\n"
        "{context_str}\n"
        "Basándote en la información de contexto proporcionada, responde la siguiente pregunta en el mismo idioma que la pregunta:\n"
        "Pregunta: {query_str}\n"
    )
    messages = [
        {"role": "system", "content": "Eres un asistente experto en Procesamiento del Lenguaje Natural. Responde siempre en el mismo idioma que la pregunta de manera útil, veraz y basada en datos."},
        {"role": "user", "content": text_qa_prompt_tmpl.format(context_str=context_str, query_str=query_str)}
    ]

    return zephyr_chat_template(messages)

def generate_response(prompt, max_new_tokens=150):
    api_key = config('HF_TOKEN', default=userdata.get('HF_TOKEN'))
    api_url = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"
    headers = {"Authorization": f"Bearer {api_key}"}
    data = {
        "inputs": prompt,
        "parameters": {
            "max_new_tokens": max_new_tokens,
            "temperature": 0.7,
            "top_k": 50,
            "top_p": 0.95
        }
    }

    response = requests.post(api_url, headers=headers, json=data)
    return response.json()[0]['generated_text'].split("Asistente: ")[-1].strip()

def chatbot(user_prompt):
    categoria = predecir_categoria(user_prompt)

    if categoria == "csv":
        context = query_tabular(user_prompt)
    elif categoria == "grafo":
        context = query_graph(user_prompt)
    else:
        context = query_chromadb(user_prompt)

    full_prompt = prepare_prompt(user_prompt, context)
    response = generate_response(full_prompt)

    return f"Prompt del usuario: {user_prompt}\n\n" \
           f"Base de datos utilizada: {categoria}\n\n" \
           f"Respuesta: {response}"

def calculate_metrics(start_time, end_time, user_prompt, response):
    response_time = end_time - start_time
    token_count = len(response.split())

    return {
        "tiempo_respuesta": response_time,
        "cantidad_tokens": token_count,
    }

if __name__ == "__main__":
    queries = [
        'Dame un ejercicio de codificación de archivos',
        '¿Cómo podría obtener texto de un archivo Word utilizando Python?',
        '¿Podrías explicarme el concepto de RAG (Retrieval Augmented Generation)?',
        '¿Que es NLP?'
    ]

    for query in queries:
        start_time = time()
        result = chatbot(query)
        end_time = time()

        print(result)
        print("\nMétricas:")
        metrics = calculate_metrics(start_time, end_time, query, result)
        for key, value in metrics.items():
            print(f"{key}: {value}")
        print("\n" + "="*50 + "\n")

Prompt del usuario: Dame un ejercicio de codificación de archivos

Base de datos utilizada: csv

Respuesta: To encode a file in Python, you can use the built-in `base64` module. Here's an example:

```python
import base64

# Open the file to be encoded
with open('file.txt', 'rb') as file:
    # Read the contents of the file
    contents = file.read()

# Encode the contents using base64
encoded = base64.b64encode(contents)

# Save the encoded file
with open('encoded_file.txt', 'wb') as encoded_file:
    encoded_file.write(encoded)
```

In this example, we first

Métricas:
tiempo_respuesta: 3.5300397872924805
cantidad_tokens: 83


Prompt del usuario: ¿Cómo podría obtener texto de un archivo Word utilizando Python?

Base de datos utilizada: csv

Respuesta: Para obtener texto de un archivo Word utilizando Python, puedes usar la biblioteca `PyWin32`. Primero, debes instalar esta biblioteca. Para ello, puedes ejecutar el siguiente comando en el terminal:

```
pip install pywin32
```

Luego, 